In [9]:
# !pip install transformers
# !pip install nltk rouge-score scikit-learn torch

  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nv

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from transformers import BertTokenizer, BertForQuestionAnswering, AdamW
from transformers import get_linear_schedule_with_warmup
import torch
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from sklearn.metrics import f1_score
from Datatests import *

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
df = pd.read_csv('/content/gdrive/My Drive/Indigo-HackToHire/processed_data.csv', index_col=0)
df.head(5)

,question,tokenized_question,stemmed_question,setmmed_nostop_answer,answer,tokenized_answer,stemmed_answer
0,Why whenever I get in the shower my girlfriend...,"['why', 'whenever', 'i', 'get', 'in', 'the', '...","['whi', 'whenev', 'i', 'get', 'in', 'the', 'sh...","['whi', 'whenev', 'get', 'shower', 'girlfriend...",Isn’t it awful? You would swear that there was...,"['isn’t', 'it', 'awful', 'you', 'would', 'swea...","['isn’t', 'it', 'aw', 'you', 'would', 'swear',..."
1,"What is a proxy, and how can I use one?","['what', 'is', 'a', 'proxy', 'and', 'how', 'ca...","['what', 'is', 'a', 'proxi', 'and', 'how', 'ca...","['proxi', 'use', 'one']",A proxy server is a system or router that prov...,"['a', 'proxy', 'server', 'is', 'a', 'system', ...","['a', 'proxi', 'server', 'is', 'a', 'system', ..."
2,"What song has the lyrics ""someone left the cak...","['what', 'song', 'has', 'the', 'lyrics', 'some...","['what', 'song', 'ha', 'the', 'lyric', 'someon...","['song', 'ha', 'lyric', 'someon', 'left', 'cak...",MacArthur's Park\n,"['macarthur', 's', 'park']","['macarthur', 's', 'park']"
3,I am the owner of an adult website called http...,"['i', 'am', 'the', 'owner', 'of', 'an', 'adult...","['i', 'am', 'the', 'owner', 'of', 'an', 'adult...","['owner', 'adult', 'websit', 'call', 'http', '...",Don't let apps that are liers put adds on your...,"['don', 't', 'let', 'apps', 'that', 'are', 'li...","['don', 't', 'let', 'app', 'that', 'are', 'lie..."
4,Does the Bible mention anything about a place ...,"['does', 'the', 'bible', 'mention', 'anything'...","['doe', 'the', 'bibl', 'mention', 'anyth', 'ab...","['doe', 'bibl', 'mention', 'anyth', 'place', '...",St. John in the book of Revelation mentions an...,"['st', 'john', 'in', 'the', 'book', 'of', 'rev...","['st', 'john', 'in', 'the', 'book', 'of', 'rev..."


In [ ]:
# Test & Train Split
df = df.sample(frac=1)

# Ratio and index definition
ratio = 0.75
total_rows = df.shape[0]
train_size = int(total_rows*ratio)

# Split data into test and train
train = df[0:train_size]
test = df[train_size:]

In [5]:
# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

# Create dataset and dataloader
dataset = BERTDataset(train, tokenizer)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# Set device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = model.to(device)

# Training parameters
epochs = 2
optimizer = AdamW(model.parameters(), lr=2e-5)
total_steps = len(dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Training loop
for epoch in range(epochs):
    model.train()
    loop = tqdm(dataloader, leave=True)

    for batch in loop:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

        loss = outputs[0].mean()

        loss.backward()
        optimizer.step()
        scheduler.step()

        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

print('Training complete!')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Streaming output truncated to the last 5000 lines.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Epoch 1:  54%|█████▍    | 1431/2644 [18:52<15:37,  1.29it/s, loss=-40.7]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some

Training complete!


In [7]:
model.save_pretrained('/content/gdrive/My Drive/Indigo-HackToHire/fine_tuned_bert_model')

In [11]:
def generate_answer(model, tokenizer, question, device='cuda'):
    inputs = tokenizer.encode_plus(question, return_tensors='pt').to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        answer_start = torch.argmax(outputs.start_logits)
        answer_end = torch.argmax(outputs.end_logits) + 1
        answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs.input_ids[0][answer_start:answer_end]))
    return answer

def compute_metrics(references, predictions):
    bleu_scores = []
    rouge_l_scores = []
    f1_scores = []

    rouge = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

    for ref, pred in zip(references, predictions):
        # BLEU score
        bleu_score = sentence_bleu([ref.split()], pred.split())
        bleu_scores.append(bleu_score)

        # ROUGE-L score
        rouge_l_score = rouge.score(ref, pred)['rougeL'].fmeasure
        rouge_l_scores.append(rouge_l_score)

        # F1-score
        ref_tokens = set(ref.split())
        pred_tokens = set(pred.split())
        common_tokens = ref_tokens.intersection(pred_tokens)

        if len(common_tokens) == 0:
            f1 = 0.0
        else:
            precision = len(common_tokens) / len(pred_tokens)
            recall = len(common_tokens) / len(ref_tokens)
            f1 = 2 * (precision * recall) / (precision + recall)
        f1_scores.append(f1)

    avg_bleu = sum(bleu_scores) / len(bleu_scores)
    avg_rouge_l = sum(rouge_l_scores) / len(rouge_l_scores)
    avg_f1 = sum(f1_scores) / len(f1_scores)

    return avg_bleu, avg_rouge_l, avg_f1

# Example usage
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

questions = test.question
references = test.answer

predictions = [generate_answer(model, tokenizer, q, device) for q in questions]

avg_bleu, avg_rouge_l, avg_f1 = compute_metrics(references, predictions)

print(f"Average BLEU Score: {avg_bleu}")
print(f"Average ROUGE-L Score: {avg_rouge_l}")
print(f"Average F1 Score: {avg_f1}")

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

Average BLEU Score: 0.00029041511713412155
Average ROUGE-L Score: 0.025171107028339794
Average F1 Score: 0.025461651211132204
